In [ ]:
# acc, wrong avg tokens, right avg tokens, avg flops, corrct_tokens_ratio

In [1]:
import __init__
from speculative.test import *
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"  # or your model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
tokenizer.pad_token = tokenizer.pad_token or tokenizer.eos_token
for n in [10, 20, 125, 250]:
    results = benchmark_final_token_latency_by_prompt_len(model, tokenizer, n_generate=n)
    print(results)

/home/wxy320/miniconda3/envs/multio1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


[{'prompt_len': 50, 'generate_n': 0.8202776908874512, 'generate_last': 0.05501508712768555, 'ratio': 14.910049837486458}, {'prompt_len': 500, 'generate_n': 0.35877037048339844, 'generate_last': 0.053021907806396484, 'ratio': 6.766455326228696}, {'prompt_len': 1024, 'generate_n': 0.45013928413391113, 'generate_last': 0.04739665985107422, 'ratio': 9.497278617276}, {'prompt_len': 4096, 'generate_n': 0.2528512477874756, 'generate_last': 0.030248641967773438, 'ratio': 8.359094205183176}, {'prompt_len': 8192, 'generate_n': 0.25091099739074707, 'generate_last': 0.030231714248657227, 'ratio': 8.29959542905813}, {'prompt_len': 12288, 'generate_n': 0.25008678436279297, 'generate_last': 0.0373539924621582, 'ratio': 6.695048316887295}, {'prompt_len': 15360, 'generate_n': 0.2595670223236084, 'generate_last': 0.0434720516204834, 'ratio': 5.970894233142293}]
[{'prompt_len': 50, 'generate_n': 0.4940340518951416, 'generate_last': 0.03111410140991211, 'ratio': 15.878139798623776}, {'prompt_len': 500, 'g

## results

In [7]:
config = configs['32b']
info = config.hidden_size, config.intermediate_size, config.num_attention_heads

p_l, l = 5000, 100
l/many_decode_flops(p_l, l, *info), l/cum_decode_flop(p_l, l, *info)

(4.193986856246504e-09, 1.048496714061626e-09)

In [1]:
from transformers import AutoConfig
configs = {
    '1b': AutoConfig.from_pretrained('Qwen/Qwen2.5-1.5B-Instruct'),
    '1b-instruct': AutoConfig.from_pretrained('Qwen/Qwen2.5-1.5B-Instruct'),
    '7b': AutoConfig.from_pretrained('Qwen/Qwen2.5-7B-Instruct'),
    '7b-instruct': AutoConfig.from_pretrained('Qwen/Qwen2.5-7B-Instruct'),
    '14b': AutoConfig.from_pretrained('Qwen/Qwen2.5-14B-Instruct'),
    '14b-instruct': AutoConfig.from_pretrained('Qwen/Qwen2.5-14B-Instruct'),
    '32b': AutoConfig.from_pretrained('Qwen/Qwen2.5-32B-Instruct'),
    '32b-instruct': AutoConfig.from_pretrained('Qwen/Qwen2.5-32B-Instruct')
}

def many_decode_flops(p_l, s, h, h_, n):
    # return 8*s*(h^2) + 16*s*h + 4*s*p_l*h + 4*s*p_l*n + 6*s*h*h_ + 2*s*h_
    return cum_decode_flop(p_l, s, h, h_, n)/5

def prefill_flop(s, h, h_, n):
    return 8*s*(h^2) + 16*s*h + 4*(s^2)*h + 4*(s^2)*n + 6*s*h*h_ + 2*s*h_

def decode_flop(s, h, h_, n):
    return 8*(h^2) + 16*h + 4*s*h + 4*s*n + 6*h*h_ + 2*h_

def cum_decode_flop(p_l, l, h, h_, n):
    flops = 0
    for i in range(l):
        flops = flops + decode_flop(p_l+i, h, h_, n)
    return flops

def get_flops(name, help_model, token_usages):
    correct_num = 0
    prefill_flops, prefix_flops, decode_flops=0,0,0
    if help_model not in configs.keys():
        config = configs[name]
        info= config.hidden_size, config.intermediate_size, config.num_attention_heads
        prefill_flops = prefill_flop(token_usages['prompt_tokens'], *info) 
        decode_flops = cum_decode_flop(token_usages['prompt_tokens'], token_usages['completion_tokens'], *info)
    else:
        spe_config = configs[name]
        tgt_config = configs[help_model]
        spe_info = spe_config.hidden_size, spe_config.intermediate_size, spe_config.num_attention_heads
        tgt_info = tgt_config.hidden_size, tgt_config.intermediate_size, tgt_config.num_attention_heads
        correct_tokens = token_usages['correct_tokens']
        completion_tokens = token_usages['completion_tokens']
        prompt_tokens = token_usages['prompt_tokens']
        last_spe_decode_pos = prompt_tokens
        token_num = 0
        prefill_flops = prefill_flops+prefill_flop(prompt_tokens, *spe_info)
        prefill_flops = prefill_flops+prefill_flop(prompt_tokens, *tgt_info)
        for cor_t in correct_tokens:
            pos, token_num = cor_t['pos']+prompt_tokens, cor_t['token_num']
            decode_flops = decode_flops+cum_decode_flop(last_spe_decode_pos, pos-last_spe_decode_pos, *spe_info)
            f = many_decode_flops(last_spe_decode_pos, pos-last_spe_decode_pos, *tgt_info)
            prefix_flops = prefix_flops+f #*f/cum_decode_flop(last_spe_decode_pos, pos-last_spe_decode_pos, *tgt_info)
            decode_flops = decode_flops+cum_decode_flop(pos, token_num, *tgt_info)
            f = many_decode_flops(pos, token_num, *spe_info)
            prefix_flops = prefix_flops+f #*f/cum_decode_flop(pos, token_num, *spe_info)
            correct_num = correct_num + token_num
            last_spe_decode_pos = pos
        decode_flops = decode_flops+cum_decode_flop(last_spe_decode_pos+token_num, completion_tokens+prompt_tokens-last_spe_decode_pos-token_num, *spe_info)
    return prefill_flops, prefix_flops, decode_flops, correct_num


/home/wxy320/miniconda3/envs/multio1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
print(name, model)
t=0.6
spe_config = configs[name]
tgt_config = configs[model]
spe_info = spe_config.hidden_size, spe_config.intermediate_size, spe_config.num_attention_heads
tgt_info = tgt_config.hidden_size, tgt_config.intermediate_size, tgt_config.num_attention_heads
for k in data.keys():
    print(data[k].keys())
    token_usages = data[k]['token_usages'][str(t)][0]
    correct_tokens = token_usages['correct_tokens']
    completion_tokens = token_usages['completion_tokens']
    prompt_tokens = token_usages['prompt_tokens']
    decode_spe_flops, decode_tgt_flops = 0, 0
    last_spe_decode_pos = completion_tokens
    correct_num, token_num = 0, 0
    print(completion_tokens, prompt_tokens )
    for cor_t in correct_tokens:
        pos, token_num = cor_t['pos']+prompt_tokens, cor_t['token_num']
        print(pos, token_num )
        decode_spe_flops = decode_spe_flops+cum_decode_flop(last_spe_decode_pos, pos-last_spe_decode_pos, *spe_info)
        decode_tgt_flops = decode_tgt_flops+cum_decode_flop(pos, token_num, *tgt_info)
        correct_num = correct_num + token_num
        last_spe_decode_pos = pos
    break
    decode_spe_flops = decode_spe_flops+prefill_flop(correct_num+prompt_tokens, *spe_info)
    decode_tgt_flops = decode_tgt_flops+prefill_flop(completion_tokens-correct_num, *tgt_info)
    print(token_usages)
    flops = decode_spe_flops+decode_tgt_flops
    print(flops)
    break

7b 32b
dict_keys(['problem', 'solution', 'answer', 'subject', 'level', 'unique_id', 'responses', 'token_usages', 'prompt', 'input_conversation'])
1534 67
696 20
716 125
991 20
1134 20


In [2]:
import os
import json

def get_info(data, name, help_model, t=0.6):
    correct_num, correct_tokens, worng_tokens, all_time_spend = 0, 0, 0, 0
    fix_tokens, prefill_flops, prefix_flops, decode_flops, = 0, 0, 0, 0
    for k in data.keys():
        # print(data[k].keys())
        token_usages = data[k]['token_usages'][str(t)][0]
        # time_spend = token_usages['time_spend']
        tokens_num = token_usages['completion_tokens']
        prefill_flop, prefix_flop, decode_flop, fix_token = get_flops(name, help_model, token_usages)
        prefill_flops = prefill_flops + prefill_flop
        prefix_flops = prefix_flops + prefix_flop
        decode_flops = decode_flops + decode_flop
        if data[k]['responses'][str(t)][0]['correctness']: 
            correct_num = correct_num+1
            correct_tokens = correct_tokens+tokens_num
        else:
            worng_tokens = worng_tokens+tokens_num
        fix_tokens = fix_tokens+fix_token
        #  all_time_spend = all_time_spend + time_spend
    all_num = len(data.keys())
    r = 10000000000
    return {
        'all_num':all_num,
        'acc':round(correct_num/all_num, 4),
        'correct_avg_tokens':round(correct_tokens/correct_num, 2) if correct_num != 0 else 0,
        'worng_avg_tokens':round(worng_tokens/(all_num-correct_num), 2) if all_num-correct_num != 0 else 0,
        'prefill_flops': round(prefill_flops/(r), 2),
        'prefix_flops': round(prefix_flops/(r), 2),
        'decode_flops': round(decode_flops/(r), 2),
        't/flops': round((correct_tokens+worng_tokens)* r/(decode_flops+prefix_flops+prefill_flops), 2),
        #  'avg_time_spend': round((correct_tokens+worng_tokens)/all_time_spend, 2),
        'fix_tokens': round(fix_tokens/(correct_tokens+worng_tokens),2)
    }

In [9]:
from collections import defaultdict
results = defaultdict(dict)

folder_path = "/home/wxy320/ondemand/program/speculative_thinking/analysis/1/normal"  # 替换为你的文件夹路径
json_files = [f for f in os.listdir(folder_path) if f.endswith(".json")]
for file in json_files:
    name = file.split('_')[0].split('-')[-1].lower()
    if name == '1.5b': name = '1b'
    dataset = file.split('_')[1]
    file_path = os.path.join(folder_path, file)
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        result = get_info(data, name, None)
    results[(name,dataset)] = {
        'normal': result
    }

In [10]:
folder_path = "/home/wxy320/ondemand/program/speculative_thinking/analysis/1/speculative"  # 替换为你的文件夹路径
json_files = [f for f in os.listdir(folder_path) if f.endswith(".json")]

for file in json_files:
    name, model, dataset = file.split('_')[0], file.split('_')[1].split('.')[0], file.split('_')[2]
    file_path = os.path.join(folder_path, file)
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        result = get_info(data, name, model)
        # print(result)
    results[(name,dataset)][model] = result

In [14]:
def convert_to_dataframe(final_results):
    import pandas as pd
    from collections import defaultdict

    # 示例数据（替换为你的数据）
    data = final_results

    # 转换数据格式
    def convert_to_dataframe(data):
        rows = []
        for dataset, models in data.items():
            for spec_model, targets in models.items():
                for target_model, metrics in targets.items():
                    row = {
                        'dataset': dataset,
                        'speculative_model': spec_model,
                        'target_model': target_model,
                    }
                    row.update(metrics)
                    rows.append(row)
        
        return pd.DataFrame(rows)

    # 转换为 DataFrame
    df = convert_to_dataframe(data)

    # 保存为 CSV 文件
    df.to_csv("final_result.csv", index=False)

    # 显示表格
    df.head()

In [11]:
final_results = defaultdict(dict)
for key, r in results.items():
    # print(key)
    final_results[key[1]][key[0]]=r
for d, r in final_results.items():
    print(d)
    print('------------------------')
    for k, r_ in r.items():
        print(k)
        for k_, r_1 in r_.items():
            print(k_, r_1)
    print('------------------------')

aime22
------------------------
1b
normal {'all_num': 30, 'acc': 0.3, 'correct_avg_tokens': 9296.56, 'worng_avg_tokens': 21558.86, 'prefill_flops': 34.75, 'prefix_flops': 0.0, 'decode_flops': 8357.59, 't/flops': 63.92, 'fix_tokens': 0.0}
14b {'all_num': 30, 'acc': 0.3333, 'correct_avg_tokens': 7319.2, 'worng_avg_tokens': 22248.6, 'prefill_flops': 207.31, 'prefix_flops': 7048.95, 'decode_flops': 14200.16, 't/flops': 24.15, 'fix_tokens': 0.17}
32b {'all_num': 30, 'acc': 0.2667, 'correct_avg_tokens': 7282.12, 'worng_avg_tokens': 18987.59, 'prefill_flops': 380.8, 'prefix_flops': 9737.43, 'decode_flops': 15750.35, 't/flops': 18.4, 'fix_tokens': 0.17}
7b
normal {'all_num': 30, 'acc': 0.5, 'correct_avg_tokens': 9178.8, 'worng_avg_tokens': 16906.67, 'prefill_flops': 171.36, 'prefix_flops': 0.0, 'decode_flops': 21454.66, 't/flops': 18.09, 'fix_tokens': 0.0}
14b {'all_num': 30, 'acc': 0.4667, 'correct_avg_tokens': 9469.93, 'worng_avg_tokens': 22208.62, 'prefill_flops': 340.02, 'prefix_flops': 69

In [15]:
from collections import defaultdict
results = defaultdict(dict)

folder_path = "/home/wxy320/ondemand/program/speculative_thinking/analysis/instruct/normal"  # 替换为你的文件夹路径
json_files = [f for f in os.listdir(folder_path) if f.endswith(".json")]
for file in json_files:
    name = file.split('_')[0].split('-')[-2].lower()
    if name == '1.5b': name = '1b'
    dataset = file.split('_')[1]
    file_path = os.path.join(folder_path, file)
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        result = get_info(data, name, None)
    results[(name,dataset)] = {
        'normal': result
    }

In [16]:
folder_path = "/home/wxy320/ondemand/program/speculative_thinking/analysis/instruct/speculative"  # 替换为你的文件夹路径
json_files = [f for f in os.listdir(folder_path) if f.endswith(".json")]

for file in json_files:
    name, model, dataset = file.split('_')[0], file.split('_')[1].split('.')[0], file.split('_')[2]
    if 'instruct' in name: name = name.split('-')[0]
    file_path = os.path.join(folder_path, file)
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        result = get_info(data, name, model)
        # print(result)
    results[(name,dataset)][model] = result

In [17]:
final_results = defaultdict(dict)
for key, r in results.items():
    # print(key)
    final_results[key[1]][key[0]]=r
for d, r in final_results.items():
    print(d)
    print('------------------------')
    for k, r_ in r.items():
        print(k)
        for k_, r_1 in r_.items():
            print(k_, r_1)
    print('------------------------')

aime22
------------------------
1b
normal {'all_num': 30, 'acc': 0.0, 'correct_avg_tokens': 0, 'worng_avg_tokens': 1954.3, 'prefill_flops': 42.19, 'prefix_flops': 0.0, 'decode_flops': 828.42, 't/flops': 67.34, 'fix_tokens': 0.0}
7b {'all_num': 30, 'acc': 0.1667, 'correct_avg_tokens': 18455.2, 'worng_avg_tokens': 14675.16, 'prefill_flops': 200.22, 'prefix_flops': 4428.0, 'decode_flops': 15974.31, 't/flops': 22.29, 'fix_tokens': 0.36}
32b {'all_num': 30, 'acc': 0.0667, 'correct_avg_tokens': 4298.5, 'worng_avg_tokens': 16168.0, 'prefill_flops': 380.8, 'prefix_flops': 8143.86, 'decode_flops': 22877.31, 't/flops': 14.69, 'fix_tokens': 0.32}
7b
normal {'all_num': 30, 'acc': 0.0667, 'correct_avg_tokens': 1052.5, 'worng_avg_tokens': 1492.54, 'prefill_flops': 208.03, 'prefix_flops': 0.0, 'decode_flops': 1925.83, 't/flops': 20.57, 'fix_tokens': 0.0}
1b {'all_num': 30, 'acc': 0.0, 'correct_avg_tokens': 0, 'worng_avg_tokens': 8559.93, 'prefill_flops': 200.22, 'prefix_flops': 2071.4, 'decode_flops'

In [18]:
convert_to_dataframe(final_results)